<p style="float: left;"><a href="_index.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="p4-exercises.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Actor model

The **actor model** is a mathematical framework for managing parallel computation in high-performance networks.

It specifically tackles challenges such as:

* Encapsulation,
* The illusion of a call stack.

These challenges arise because **traditional programming assumptions no longer align with the realities of modern multi-threaded, multi-CPU systems.**

## The Problem of distributed domputation and object-oriented programming (OOP)

- **One of the OPP core pillars  is _encapsulation_.**
  
- Encapsulation dictates that **the internal data of an object is not accessible directly from the outside**; it can only be modified by invoking a set of curated methods. 

- The object is responsible for exposing safe operations that protect the invariant nature of its encapsulated data.

    ![](https://getakka.net/images/seq_chart.png)
    _Runtime behaviour. Interactions of method calls._

    - **A _thread_ executes all the method's calls, and the enforcement of invariants occurs on the same thread**
    from which the method was called.
    
        ![](https://getakka.net/images/seq_chart_thread.png)
        _Executions accurs on the same thread._

    - **In multiple thread system, the invariance might not be respected.**

      ![](https://doc.akka.io/libraries/akka-core/current/typed/guide/diagrams/seq_chart_multi_thread.png)
        _Method's executions in a multi-thred system._

        - **The encapsulation model of objects does not guarantee anything about what happens where two threads enter the same method**.
                    
        - **The problem appears because objects share mutable data between them (data raise) and <span style="color: red">you are not suppose to abstract over data raises**.</span>

        - Instructions of the two invocations can be interleaved in arbitrary ways which eliminate **any hope for keeping the invariants intact without some type of coordination.**

        - A common approach to solving this problem is to add a lock around these methods.

            - Locks are very costly on modern CPU architectures, requiring heavy-lifting from the operating system to suspend the thread and restore it later.

            - The caller thread is now blocked, so it cannot do any other meaningful work.
    
            - Locks introduce a new menace: deadlocks.

            - When it comes to coordinating across multiple machines, the only alternative is distributed locks. <span style="color:red">**Distributed locks are several magnitudes less efficient than local locks**. </span>

## Call stacks and tasks delegation

- **A call stack is a stack data structure that stores information about the active subroutines of a computer program.**
      
- Call stacks seems to be a good way to operate between threds.


- **They don’t handle asynchronous operations well because they don’t cross thread boundaries <span style="color:red">(isolated)</span>**

    - **Completion notification**: How does the "caller" thread know the task is complete in the "worker" thread?
 
    - **Exception handling**: If an error occurs, the exception goes to the worker’s handler, bypassing the caller. Without a direct way to notify the caller.
 
    - Messages can be lost. This leaves the system without a way to recover the lost task, even in local (non-networked) communication.
         
      ![](https://getakka.net/images/exception_prop.png)

-  <span style="color:red">Concurrent systems must handle service faults and provide recovery mechanisms.</span>


## Actors for distributed systems

### What is an actor?

- An actor as the basic **building block of concurrent computation.**
  
- **It encapsulates computation and holds its own private state**
  
- **Actors communicates with other actors solely by passing asynchronous messages.**
  
- **Actors have a mailbox where messages arrive and are processed at most one at a time** in the order they are received.
  
- **How an actor responds / reacts to a message essentially defines its behavior.**

- Essenciatily, an actor can:
  
    1) Send messages to other actors,
       
    2) Respond to received messages,
       
    3) Create new actors.


![](https://getakka.net/images/serialized_timeline_invariants.png) _Exemple of asynchronous messages between actors_

### Actor's hierarchical structure

- **Actors are hierarchy organized.**

    - An actor responsible for a particular function might choose to break down into smaller manageable segments.

    - Creating child actors, over which it maintains control.
 
- **Each actor has one supervisor, the actor that initiated its creation.**
  
- **When an actor is unable to address a certain situation, it sends a failure message to its supervisor**, requesting assistance.

- **This recursive setup enables failure handling at the appropriate level in the hierarchy.**

![](https://getakka.net/images/actor_tree_supervision.png) _Actor's hierarchy. Failure handling_

## Actors & SCALA

In [ ]:
import $ivy.`com.typesafe.akka::akka-actor:2.6.10`

In [1]:
scala.util.Properties.versionString

res1: String = "version 2.13.16"


- We will use the [Akka actors](https://doc.akka.io/libraries/akka-core/current/typed/actors.html#akka-actors) of the [Akka platform](https://github.com/akka/akka).

- Akka’s actor model provides an abstraction that simplifies writing correct concurrent, parallel, and distributed systems.

- The Akka platform offers:

  - **Akka actors** – toolkit for building concurrent and distributed applications
    
  - **Concurrency** – safe and efficient message-driven execution
    
  - **Scalability** – scale locally with threads or across nodes with remote actors
    
  - **Fault tolerance** – supervision and self-healing mechanisms
    
  - **Unified programming model** – same abstraction for local and distributed actors
    
  - **Integrated runtime management** – monitoring and deployment tools
    
  - **Open-source** – community-driven and widely adopted
 
- <span style="color:red">**We will use the object-oriented style.**</span> **Where a concrete class for the actor behavior is defined and mutable state is kept inside of it as fields.**

### Defining an actor

- You can create an actor extending a class from the `Actor` class

    ```scala
    class Master extends Actor {
        . . .
    }
    
    ```

    <br/>

- The `Akka Actor` has a key lifecycle hooks like `preStart`, `postStop`, `preRestart`, and `postRestart` that can be overrided.

- The lifecycle of a child actor is tied to the parent – **a child can stop itself or be stopped at any time but it can never outlive its parent.**

- Each actor must define the partial method `receive`, it defines the behaviour of the actor.

### Context and actor factory

- **Actor context** - the view of the actor cell from the actor.

  - Exposes contextual information for the actor and the current message.
 
    ```scala
    context.self // Self reference of the actor
    
    ```
    
    <br/>
    
  - Let us create other actors:

    ```scala
    context.actorOf(Props[MyActor]) // : ActorRef
    context.actorOf(Props(classOf[MyActor], arg1, arg2), "name") // : ActorRef
    
    ```

<br/>

- **Actor system** - a hierarchical group of actors which share common configuration.

    ```scala
    system.actorOf(Props[MyActor], "name") // : ActorSystem
    system.actorOf(Props(classOf[MyActor], arg1, arg2), "name") // : ActorSystem

    ```

### Receiving messages

- In classic Akka, **each actor’s behavior is defined by the partial function returned by the method `receive`.**
  
    ```scala
    class Talker extends Actor {

        . . .
        
        def receive: Receive = {
            case Greet(name)          => greet(name)
            case Praise(name)         => praise(name)
            case Celebrate(name, age) => celebrate(name, age)
        }
    
        . . .
    }
    
    ```

### Sending messages

- Messages contain objects of any type, but they must be immutable.

- `case class` objects *fit like a glove* for messaging purposes.

  - They are immutable.
    
  - They have semantic meaning.

- Types of messages:

    - **tell (!)**: a "fire-and-forget" approach, where **a message is sent asynchronously without waiting for a response.**
      
    - **ask (?)**: sends a message asynchronously but returns a `Future` representing a possible reply from the receiver.
     
        - You can "subscribe" to the future, allowing the thread to remain idle until the result is ready <span style="color:red">**(We will see this latter)**</span>
        
        <br/>
  
    - The sender can include its own `ActorRef`, allowing the receiving actor to respond directly to the sender.

- **The only way to communicate with an actor is through messaging.** <span style="color:red">**(You don’t call methods on actors)**</span> 

Example:

```scala
case class Greet(name: String, ar: ActorRef)
case class Praise(name: String)
case class Celebrate(name: String, age: Int)

class Master extends Actor {
    val talker: ActorRef = context.actorOf(Props(new Talker), "talker") // Actor, creates another actor to communicate with
    
    override def preStart(): Unit =  {
        context.watch(talker)
        talker ! Greet("Huey", self) // Sending self actorRef
        talker ! Praise("Dewey")
        talker ! Celebrate("Louie", 16)
    }

    . . .
}

```

### Demo actors

<span style="color:red">**This example is using Scala 2.13.**</span>

- Installing the `Akka actor` library.

In [ ]:
import $ivy.`com.typesafe.akka::akka-actor:2.6.10`

import akka.actor.{Actor, ActorRef}
import akka.actor.{ActorSystem, PoisonPill, Props}
import akka.actor.{Actor, ActorRef, ActorSystem, PoisonPill, Props, Terminated}

- Defining the `Talker` actor.

In [ ]:
case class Greet(name: String, ar: ActorRef)
case class Praise(name: String)
case class Celebrate(name: String, age: Int)

class Talker extends Actor {
    var nMessages = 0
    val limitMessages = 2
    
    def receive: Receive  = {
        case Greet(name, ar)      => greet(name, ar)
        case Praise(name)         => praise(name)
        case Celebrate(name, age) => celebrate(name, age)
    }

    def updateMessages(n: Int): Unit = {
        nMessages += n
        println(s"> updateMessages: nMessages: $nMessages")

        if (nMessages >= limitMessages)
            stopSelf()
    }

    private def stopSelf(): Unit = {
        println("> stopSelf: stopping talker actor")
        context.stop(self)
    }

    def greet(name: String, ar: ActorRef): Unit = {
        println(s"> Greet($name, $ar): Hello $name from ${ar.toString}" )
        updateMessages(1)
    }

    def praise(name: String): Unit = {
        println(s"> Praise($name): $name, you're amazing")
        updateMessages(1)
    }

    def celebrate(name: String, age: Int): Unit = {
        println(s"> Celebrate($name, $age): Here's to another $age years, $name")
        updateMessages(1)
    }
}

- Defining the `Master` actor.

In [ ]:
class Master extends Actor {
    val child: ActorRef = context.actorOf(Props(new Talker), "talker") // Creates the talker actor...
    
    override def preStart(): Unit =  {
        context.watch(child)                           // Watch talker actor child
        talker ! Greet("Huey", self)
        talker ! Praise("Dewey")
        talker ! Celebrate("Louie", 16) 
    }

    def receive: Receive = {
        case Terminated(ref) if ref == child   => {    // Child whatched has been terminated...
            println("> Terminated(`talker`): talker terminated, finishing system...")
            context.system.terminate()                 // Terminate the system
        }
    }
}

In [ ]:
object Main {
    def main(args: Array[String]): Unit = {
        val system: ActorSystem = ActorSystem("HelloActors")
        val master: ActorRef = system.actorOf(Props(new Master), "master")

        Thread.sleep(5000)   // This is arbitrary...

       // system.terminate() // Do we need this?
        println("> After sleep...")
    }
}

- Executing the program...

In [73]:
Main.main(Array(""))

> Greet(Huey, Actor[akka://HelloActors/user/master#1286490416]): Hello Huey from Actor[akka://HelloActors/user/master#1286490416]
> updateMessages: nMessages: 1
> Praise(Dewey): Dewey, you're amazing
> updateMessages: nMessages: 2
> stopSelf: stopping talker actor
> Terminated(`talker`): talker terminated, finishing system...
[INFO] [akkaDeadLetter][09/18/2025 08:52:41.597] [HelloActors-akka.actor.default-dispatcher-7] [akka://HelloActors/user/master/talker] Message [ammonite.$sess.cmd55$Helper$Celebrate] from Actor[akka://HelloActors/user/master#1286490416] to Actor[akka://HelloActors/user/master/talker#629864955] was not delivered. [1] dead letters encountered. If this is not an expected behavior then Actor[akka://HelloActors/user/master/talker#629864955] may have terminated unexpectedly. This logging can be turned off or adjusted with configuration settings 'akka.log-dead-letters' and 'akka.log-dead-letters-during-shutdown'.
> After sleep...


**NOTES:**

- The previous example might be misleading because it ran on a single-threaded container.

  - In that case, it looks as if everything happens **step by step** in a nice, predictable order.
    
  - But in a real multi-threaded system, messages are like **letters arriving at a busy post office**: they may come in a different order than you expect.
    
  - The key point is that **the order of arrival and execution is not guaranteed**.

  - **The atomic one-by-one processis is ensured** though.

# Activity ✋

Let's work witht the repo [actors-nightmare](https://github.com/wilberquito/typed-actors-nightmare)

- Open the project in your PC.

<p style="float: left;"><a href="_index.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="p4-exercises.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>